## Helical Axis Model

$$ R = R_0 + \rho\cos\theta + \delta\cos(N\zeta) $$
$$ \phi = -\zeta $$
$$ Z = \rho\sin\theta + \delta\sin(N\zeta) $$

In [ ]:
import dcs
import dcs.toroidalField
import dcs.geometry


import numpy as np
try:
    import matplotlib
    matplotlib.rcParams['text.usetex'] = True
except:
    pass
import matplotlib.pyplot as plt 


In [ ]:
r0 = 2 
rho = 0.02
delta = 0.02
nfp = 100

funR = lambda thetaArr, zetaArr: r0 + rho*np.cos(thetaArr) + delta*np.cos(nfp*zetaArr) 
funZ = lambda thetaArr, zetaArr: rho*np.sin(thetaArr) + delta*np.sin(nfp*zetaArr)

In [ ]:
sampleNumsTheta, sampleNumsZeta = 4, 4
deltaTheta = 2*np.pi / (2*sampleNumsTheta+1)
deltaZeta = 2*np.pi / nfp / (2*sampleNumsZeta+1)
sampleTheta, sampleZeta = np.arange(2*sampleNumsTheta+1)*deltaTheta, np.arange(2*sampleNums+1)*deltaVarPhi
gridSampleVarPhi, gridSampleTheta = np.meshgrid(sampleVarPhi, sampleTheta)
sampleR = funR(gridSampleTheta, -gridSampleVarPhi)
sampleZ = funZ(gridSampleTheta, - gridSampleVarPhi)

fieldR = lec.toroidalField.fftToroidalField(sampleR, nfp=nfp)
fieldZ = lec.toroidalField.fftToroidalField(sampleZ, nfp=nfp)

surfHelicalAxis = lec.geometry.Surface(fieldR, fieldZ)